In [ ]:
import numpy as np

lst = np.random.randint(0,100,53).tolist()

In [ ]:
from math import floor
col = floor(len(lst)/5)
remaining = len(lst)%5

A = [lst[i:i+5] for i in range(0,col*5,5)]
A.append(lst[col*5:])
A

def internal_lst_sort(A):
    median_lst = []
    for e in A:
        insertion_sort(e)
        median_lst.append(e[floor(len(e)/2)])
    return median_lst

median_lst = internal_lst_sort(A)

median_lst

In [ ]:
from math import ceil
def partition(A,p,r):
    x = A[r]
    i = p-1
    for j in range(p,r):
        if A[j] <= x:
            i += 1
            A[i],A[j] = A[j],A[i]
    A[i+1],A[r] = A[r],A[i+1]
    return i+1,A[i+1]

def median_finding(A,n,i):
    k, x = partition(A,0,n-1)
    L = A[:k]
    R = A[k+1:]
    print(f"k:{k}, i:{i}, n:{n}, A:{A}, L:{L}, R:{R}, pivot:{x}")
    if i == k:
        return x
    elif i < k:
        return median_finding(L,k,i) # recurse at left
    else:
        return median_finding(R,n-k,i-k-1) # recurse at right; Why (i-k)??? because recure on the right part whose value
                                    #is greater than x

median_lst = [2,8,7,1,3,5,6,9,4]

median_finding(median_lst,len(median_lst),5)

## Divide and Conquer (FFT)

#### Recursive

In [ ]:
# Step 1: Convert a,b coefficient to sample using DFT
# Step 2: Muntiply element-wise c = a*b
# Step 3: Covert c back to coefficient representation


from cmath import exp, pi

def recursive_fft(a):

# Base case
    n = len(a)
    if n == 1:
        return a
    omega_n = exp(2j*pi/n) # Principla nth root of unity
    omega = 1
    y = [0]*n

# Divide step
    a_0 = a[::2] # even subproblem
    a_1 = a[1::2] # odd subproblem

# Conquer step
    y_0 = recursive_fft(a_0)
    y_1 = recursive_fft(a_1)

# Combine step (A(x) = A_0[x^2] + x*A_1[x^2])
    for k in range(0,n//2):
        y[k] = y_0[k] + omega*y_1[k]
        y[k+n//2] = y_0[k] - omega*y_1[k]
        omega = omega*omega_n
    return y

a = [9, -10, 7, 6, 0, 0, 0, 0]
b = [-5, 4, 0, -2, 0, 0, 0, 0]
sample_a = recursive_fft(a)
sample_b = recursive_fft(b)
sample_c = [sample_a[i]*sample_b[i] for i in range(len(a))]
sample_a

In [ ]:
def unscale_rifft(a):
    n = len(a)
    if n == 1:
        return a
    omega_n = exp(-2j*pi/n)
    omega = 1
    y = [0]*n

    a_0 = a[0::2]
    a_1 = a[1::2]

    y_0 = unscale_rifft(a_0)
    y_1 = unscale_rifft(a_1)

    for k in range(0,n//2):
        y[k] = y_0[k] + omega*y_1[k]
        y[k+n//2] = y_0[k] - omega*y_1[k]
        omega = omega*omega_n
    return y

def rifft(a):
    a = unscale_rifft(a)
    for i in range(len(a)):
        a[i] = a[i]/len(a)
    return a

result = [round(e.real,2) for e in rifft(sample_c)]
result

#### Bottom Up

In [ ]:
def bit_reversal_copy(a):
    n = len(a)
    A = [0]*n
    for k in range(n):
        b = '{0:0{width}{base}}'.format(k, base='b', width=3)
        rev_k = int('{0:0{width}{base}}'.format(int(b[::-1],2), base='d', width=3))
        A[rev_k] = a[k]
    return A

a = [9, -10, 7, 6, 0, 0, 0, 0]
bit_reversal_copy(a)

In [ ]:
from cmath import exp, pi
import math

def iterative_fft(a):
    A = bit_reversal_copy(a)
    n = len(A)
    lg_n = int(math.log(n,2))
    for s in range(1,lg_n+1):
        m = 2**s
        omega_m = exp(-2j*pi/m) ### Why minus
        for k in range(0,n,m): ### Why
            omega = 1
            for j in range(m//2):
                t = omega*A[k+j+m//2]
                u = A[k+j]
                A[k+j] = u + t
                A[k+j+m//2] = u - t
                omega = omega*omega_m
                # print(f's: {s}, k: {k}, j: {j}, k+j: {k+j}, k+j+m/2: {k+j+m//2}, A[k+j]: {A[k+j]}, A[k+j+m/2]: {A[k+j+m//2]}')
    return A

def bit_reversal_copy(a):
    n = len(a)
    A = [0]*n
    for k in range(n):
        b = '{0:0{width}{base}}'.format(k, base='b', width=3)
        rev_k = int('{0:0{width}{base}}'.format(int(b[::-1],2), base='d', width=3))
        A[rev_k] = a[k]
    return A

a = [9, -10, 7, 6, 0, 0, 0, 0]
iterative_fft(a)

In [ ]:
def nextpower2(i):
    n = 2
    while n < i: n = n * 2
    return n

def bitrev(x):
    #return x[::-1]
    N = len(x)
    if N != nextpower2(N): 
        print("error")
        exit(0)

    x = x[:]
    for i in range(N):
        k, b, a = 0, N>>1, 1
        while b >= a:
            if b&i: k = k|a
            if a&i: k = k|b
            b, a = b>>1, a<<1
        if i < k:               
            x[i], x[k] = x[k], x[i]
    return x

def iterative_fft_github(x):
    n = len(x)
    X = []
    x = bitrev(x)
    for i in range(n):      
        X.append(exp(-2j * pi * i /n))
    m = 2
    while m <= n:
        for s in range(0,n, m):
            for i in range(int(m/2)):
                p = i *int(n/ m)
                a = s + i
                b = s + i + int(m/2)
                x[a], x[b] = x[a] + X[p%n] * x[b], x[a] - X[p%n] * x[b]
        m = m * 2
    return x

a = [9, -10, 7, 6, 0, 0, 0, 0]
iterative_fft_github(a)

In [ ]:
a = [9, -10, 7, 6, 0, 0, 0, 0]
np.fft.fft(a), 

In [ ]:
import cmath
u = 256
pow = abs((log(u,2)).real/2)
2**floor(pow) * 2**ceil(pow)
# floor(abs(sqrt(u)))*math.ceil(abs(sqrt(u)))